 We are going to build a MLP relatively simple, but without using Keras or TF, coding our own tools to do it, in order to fully understand what's behind.

 3 dimesions entry
 1 hidden layer with 2 neurons
 1 output (binary classif)
 Relu and Sigmoid as activation functions.
 MSE as Loss function

In [ ]:
import numpy as np
import pandas as pd
import random


In [ ]:
# Initialize

# N = 5 Examples (batch size)
# d = 3, input dim
# o = 1, output dim
# h = 2, neurons in hidden layer

# training epochs
epochs = 1000

# Input (N,d)
X = np.array([[0.1, 0.5, 0.3],
              [0.4, 0.2, -0.7],
              [0.7, 0.8, 0.2],
              [-0.3, 0.9, 0.1],
              [0.6, 0.3, -0.4]])

# output (N,o)
Y = np.array([[1], [0], [1], [1], [0]]) # !!!very important to be in this format (5,1) and not the (5,) given by Y = np.array([1, 0, 1, 1, 0])

# Weights (d,h)
# Initialisation correcte des poids et biais
W1 = np.random.rand(3, 2)  # d=3, h=2
B1 = np.random.rand(1, 2)  # (1, 2) pour que le biais soit diffusé sur chaque exemple



# First activation fun
def relu(z):
  return np.maximum(z,0) #np.maximum apply the ReLU activation element-wise across the matrix, max doesn't

# Propagation avant (forward propagation)
Z1 = np.dot(X, W1) + B1  # X: (5, 3), W1: (3, 2), B1: (1, 2)
A1 = relu(Z1)  # A1: (5, 2)


W2 = np.random.rand(2, 1)  # h=2, o=1
B2 = np.random.rand(1, 1)  # (1, 1) pour le biais de sortie

Z2 = np.dot(A1, W2) + B2  # A1: (5, 2), W2: (2, 1), B2: (1, 1)



# Activation func 2 , sigmoid
def sigmoid(z):
  return (1/(1+ np.exp(-z))) #NumPy applique l'opération élément par élément (element-wise) à toute la matrice ou le vecteur, donc pas de pb ici

A2 = sigmoid(Z2)  # A2: (5, 1)

E = Y - A2 # Brute error

# Loss function
def loss(Y_true,Y_pred):
  N = Y_true.shape[0]
  compute = (1/N)*np.sum((Y_true - Y_pred)**2) # np.mean() does the exact same
  return compute




# So now we have the complete architecture of our neural network
# we need to train it on epochs = 1000 for example.

for epoch in range(epochs):

  Z1 = np.dot(X, W1) + B1
  A1 = relu(Z1)
  Z2 = np.dot(A1, W2) + B2
  A2 = sigmoid(Z2)
  loss_value = loss(Y, A2)
  # Let's compute the backpropagation
  # We already computed the formulas thanks to chain partial derivatives

  # dLoss/dB2
  dLoss_dB2 = np.sum((A2 - Y) * (A2 * (1 - A2)), axis=0, keepdims=True) # axis = 0 gives the sum along the rows,
  #meaning the results at the end of each column, keeping the number of columns so the result is
  # [sum C1, sum C2] instead of the global sum


  #dLoss/dW2
  dLoss_dW2 = np.dot(A1.T, (A2 - Y) * (A2 * (1 - A2)))


  #dLoss/dB1
  dLoss_dB1 = np.sum(((A2 - Y) * (A2 * (1 - A2))) @ W2.T * (Z1 > 0), axis=0, keepdims=True)


  #dLoss/dW1
  dLoss_dW1 = np.dot(X.T, ((A2 - Y) * (A2 * (1 - A2))) @ W2.T * (Z1 > 0))

  # at each epochs we update the parameters
  learning_rate = 0.1

  # Correct going in the opposite direction of the gradient
  B1 -= learning_rate * dLoss_dB1
  B2 -= learning_rate * dLoss_dB2
  W1 -= learning_rate * dLoss_dW1
  W2 -= learning_rate * dLoss_dW2

  # Affichage de la perte toutes les 100 époques
  if epoch % 100 == 0:
      print(f"Epoch {epoch}, Loss: {loss_value}")



Epoch 0, Loss: 0.24061956601979217
Epoch 100, Loss: 0.1017810053666683
Epoch 200, Loss: 0.0298923598931502
Epoch 300, Loss: 0.014349203383598096
Epoch 400, Loss: 0.00890701244973803
Epoch 500, Loss: 0.00630130368467822
Epoch 600, Loss: 0.004813478701372741
Epoch 700, Loss: 0.0038637582697041957
Epoch 800, Loss: 0.003211078173272466
Epoch 900, Loss: 0.0027375970347008442


Entrée : 3 neurones (dimensions des entrées).
Deux couches cachées : 1ère couche avec 4 neurones, 2ème couche avec 3 neurones.
Sortie : 2 neurones (au lieu d'une sortie binaire, on suppose ici une sortie pour une classification multi-classe).
Activation : ReLU pour les couches cachées, Softmax pour la sortie (cas de classification multi-classe).

In [ ]:

#On est dans une classif multiclasse (2 classes) ce qui est différent d'une classif binaire. En effet la sortie est forcément
#du type Y = [[1,0],[1,0],[1,0],[0,1],[1,0],[0,1]]   [1,0] et [0,1] sont les encodages oneHot de 1 et 0, nos deux classes.


########## Pourquoi Softmax exige un encodage one-hot ([1,0] et [0,1]) et empeche l'utilisation plus simple binaire de 0 ou 1  ?

#Si on a deux classes, la sortie du réseau sera un vecteur à 2 dimensions, comme ceci :

#A3 = softmax(Z3)
#où A3 pourrait donner un résultat comme :
#[[0.85, 0.15],  # Probabilité d'appartenir à la classe 0 = 85%, classe 1 = 15%
# [0.20, 0.80],  # Probabilité d'appartenir à la classe 0 = 20%, classe 1 = 80%
# ...]
#Dans ce cas, la classe prédite est l'indice du plus grand score (argmax(A3, axis=1)).

#Pourquoi on encode les labels (Y) en one-hot ([1,0] et [0,1]) ?

#Parce que Softmax prédit un vecteur et la perte de cross-entropie catégorielle (categorical_crossentropy) attend un vecteur de même dimension
#pour comparer chaque probabilité.
#Problème si Y est encodé avec [0] et [1] au lieu de one-hot :
#Avec Softmax, Y doit avoir la même forme que la sortie du réseau (A3).
#Si Y était sous la forme :
#Y = np.array([[0], [1], [1], [0], [1]])
#alors la perte essayerait de comparer un vecteur (A3 de shape (N,2)) avec un scalaire (Y de shape (N,1)), ce qui ne fonctionnerait pas.



In [ ]:
N = 10
d = 3
h1 = 4
h2 = 3
o = 2

def relu(z):
  return np.maximum(0,z)

def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # stabilité numérique
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

# X (N,d) = (5,3)
X = np.array(np.random.rand(5,3))

#Y (N,o) = (5,2)
Y = np.array([[1,0],
              [0,1],
              [0,1],
              [0,1],
              [1,0],])
# W1 (d,h1)
W1 = np.random.rand(d,h1)
# B1 (1,h1)
B1 = np.random.rand(1,h1)

#W2 (h1,h2)
W2 = np.random.rand(h1,h2)
#B2 (1,h2)
B2 = np.random.rand(1,h2)

#W3 (h2,o)
W3 = np.random.rand(h2, o)
#B3 (1,o)
B3 = np.random.rand(1,o)

# First hidden layer
Z1 = np.dot(X,W1) + B1
A1 = relu(Z1)

#Second
Z2 = np.dot(A1,W2) + B2
A2 = relu(Z2)

# Output classifiction
Z3 = np.dot(A2, W3) + B3
A3 = softmax(Z3)

Error = Y - A3

def categorical_crossentropy(Y, A): # Check l'exemple apres le training
    return -np.mean(np.sum(Y * np.log(A + 1e-9), axis=1))  # Ajout de 1e-9 pour éviter log(0)



In [ ]:
# Entraînement du réseau
epochs = 1000
learning_rate = 0.1
for epoch in range(epochs):
    # Forward propagation
    Z1 = np.dot(X, W1) + B1
    A1 = relu(Z1)
    Z2 = np.dot(A1, W2) + B2
    A2 = relu(Z2)
    Z3 = np.dot(A2, W3) + B3
    A3 = softmax(Z3)

    # Calcul de la perte
    loss_value = categorical_crossentropy(Y, A3)

    # Backpropagation
    # La seul calcul vraiment difficile est celui de la dérivée de la categorical crossentropy
    # heuresuement, elle se simplifie avec la softmax (et A3 = softmax(Z3)), d'où:
    dLoss_dZ3 = A3 - Y # La fonction de perte utilisée est l'entropie croisée avec le softmax. Le gradient de la perte par rapport à
#Z3 est assez simple grâce à la propriété du softmax combinée avec l'entropie croisée => A3 - Y
    dLoss_dW3 = np.dot(A2.T, dLoss_dZ3) # dloss/dz3 .dZ3/dw3
    dLoss_dB3 = np.sum(dLoss_dZ3, axis=0, keepdims=True)

    dLoss_dZ2 = np.dot(dLoss_dZ3, W3.T) * (Z2 > 0)
    dLoss_dW2 = np.dot(A1.T, dLoss_dZ2)
    dLoss_dB2 = np.sum(dLoss_dZ2, axis=0, keepdims=True)

    dLoss_dZ1 = np.dot(dLoss_dZ2, W2.T) * (Z1 > 0)
    dLoss_dW1 = np.dot(X.T, dLoss_dZ1)
    dLoss_dB1 = np.sum(dLoss_dZ1, axis=0, keepdims=True)

    # Mise à jour des poids et biais
    W1 -= learning_rate * dLoss_dW1
    B1 -= learning_rate * dLoss_dB1
    W2 -= learning_rate * dLoss_dW2
    B2 -= learning_rate * dLoss_dB2
    W3 -= learning_rate * dLoss_dW3
    B3 -= learning_rate * dLoss_dB3

    # Affichage de la perte toutes les 100 époques
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss_value}")


Epoch 0, Loss: 0.7119652245840109
Epoch 100, Loss: 0.8744412539910524
Epoch 200, Loss: 0.02072969118628506
Epoch 300, Loss: 0.004293450260957775
Epoch 400, Loss: 0.00183796248045954
Epoch 500, Loss: 0.0010669119867656202
Epoch 600, Loss: 0.0007217572689745832
Epoch 700, Loss: 0.000533488041412589
Epoch 800, Loss: 0.0004171166324759133
Epoch 900, Loss: 0.00033987055940390626


In [ ]:
#############  Exemple de calcul a la main avec la categorical_crossentropy
import numpy as np

# Labels one-hot (3 exemples, 2 classes)
Y_test = np.array([
    [1, 0],  # Classe 0
    [0, 1],  # Classe 1
    [1, 0]   # Classe 0
])

# Prédictions Softmax (probas pour chaque classe)
A = np.array([
    [0.9, 0.1],  # Probabilité de la classe 0: 90%, classe 1: 10%
    [0.3, 0.7],  # Probabilité de la classe 0: 30%, classe 1: 70%
    [0.8, 0.2]   # Probabilité de la classe 0: 80%, classe 1: 20%
])

# Calcul de la categorical crossentropy
loss_values = -np.sum(Y_test * np.log(A), axis=1)  # Somme sur j (les classes)
loss_mean = np.mean(loss_values)  # Moyenne sur i (les exemples)

#Y_test * np.log(A):
[[1, 0],     * log([[0.9, 0.1]])  → [log(0.9), 0]
 [0, 1],       log([[0.3, 0.7]])  → [0, log(0.7)]
 [1, 0]]       log([[0.8, 0.2]])  → [log(0.8), 0]

#= [[-0.105, 0], [0, -0.357], [-0.223, 0]]
# ensuite On somme sur axis=1 (les classes) pour obtenir un vecteur (perte pour chaque exemple)  :
# -np.sum(Y_test * np.log(A), axis=1
#[-0.105, -0.357, -0.223]
#On fait la moyenne sur tous les exemples :
#-np.mean(np.sum(Y * np.log(A + 1e-9), axis=1))
#(-0.105 - 0.357 - 0.223) / 3 ≈ 0.228

